# Script for running fineSTRUCTURE and PLINK IBS

## Before you begin

In [ ]:
cd star_protocols_saimaa
work=$(pwd)

apps=$work/bin

plink=$apps/plink

## fineSTRUCTURE

In [ ]:
#Prepare .phase files for fineSTRUCTURE

VCF=$work/processed_data/2.masking-filtering/saimaa_20_phased_posmask.vcf.gz
DIR=$work/processed_data/4.finestructure/for_fs
mkdir -p $DIR

fs_dir=$apps/fs_4.1.1


for ctg in $(head -20 $refs.fa.fai | cut -f1); do
    num=${ctg:0:6}

    $apps/plink --vcf $VCF --make-bed --chr $ctg --allow-extra-chr \
      --out $DIR/saimaa_$num --recode ped 12
    perl $fs_dir/plink2chromopainter.pl -p=$DIR/saimaa_$num.ped \
      -m=$DIR/saimaa_$num.map -o=$DIR/saimaa_$num.phase
    perl $fs_dir/makeuniformrecfile.pl $DIR/saimaa_$num.phase $DIR/saimaa_$num.rec

    rm $DIR/saimaa_$num.[blmfp]?? $DIR/saimaa_$num.nosex
done


In [ ]:
#Prepare first command


OUT=$work/processed_data/4.finestructure/

cat $work/data/saimaa_IDs.txt | sed 's/^/s/' > $OUT/saimaa.ids

cut -c-6 <(head -20 $refs.fa.fai) > $OUT/20ctgs.txt

cd $OUT
#ln -s ../$DIR data

(echo -n "fs saimaa_20ctgs.cp -idfile saimaa.ids -phasefiles " && \
    cat 20ctgs.txt | while read a; do echo -n " for_fs/saimaa_"$a".phase"; done && \
    echo -n " -recombfiles " && \
    cat 20ctgs.txt | while read a; do echo -n " for_fs/saimaa_"$a".rec"; done && \
    echo " -go") > command_20ctgs.txt


#Run the resulting command_20ctgs.txt 

source command_20ctgs.txt

#Copy outputfiles to the "results" directory

cp saimaa_20ctgs_linked.chunkcounts.out $work/results/4.finestructure/
cp saimaa_20ctgs_linked_tree.xml $work/results/4.finestructure/

## Identity-by-state using PLINK

In [ ]:
out=$work/processed_data/4.finestructure/IBS

mkdir -p $out && cd $out

$plink --vcf $VCF --allow-extra-chr --distance 'ibs' 'square'